**Implement Batch Gradient Descent with early stopping for Softmax Regression without using Scikit-Learn**

#### Loading the libraries and the IRIS dataset

In [1]:
import numpy as np
import math
from sklearn import datasets
iris = datasets.load_iris()

#### Split train, validation and test set

In [2]:
X_raw = iris.data[:, 2:] # Only get pedal width and height
y_raw = iris.target

X_with_bias = np.c_[np.ones((len(X_raw), 1)), X_raw];

np.random.seed(3433)
random_indexes = np.random.permutation(len(X_raw))

val_ratio = 0.2
test_ratio = 0.2
val_num = math.floor(val_ratio * len(X_raw))
test_num = math.floor(test_ratio * len(X_raw))
train_num = len(X_raw) - val_num - test_num

X_train = X_with_bias[random_indexes[:train_num]]
y_train = y_raw[random_indexes[:train_num]]
X_val = X_with_bias[random_indexes[train_num:-test_num]]
y_val = y_raw[random_indexes[train_num:-test_num]]
X_test = X_with_bias[random_indexes[-test_num:]]
y_test = y_raw[random_indexes[-test_num:]]


#### Data Cleansing

In [3]:
def one_hot(arr):
    classes = len(np.unique(arr))
    result = np.zeros((len(arr), classes))
    result[np.arange(len(arr)), arr] = 1
    return result

y_train_one_hot = one_hot(y_train)
y_val_one_hot = one_hot(y_val)
y_test_one_hot = one_hot(y_test)

#### Batch Gradient Descent

In [4]:
# Cross Entropy Cost Function
def x_entropy(prob_matrix, expected_matrix, epsilon):
    m = len(prob_matrix)
    k = prob_matrix.shape[1]
    total_error = 0

    for i in range(m):
        tmp = 0
        for j in range(k):
            tmp += (expected_matrix[i][j] * np.log(prob_matrix[i][j] + epsilon))
        total_error += tmp
    return -total_error / m
#     return -np.mean(np.sum(expected_matrix * np.log(prob_matrix + epsilon), axis=1))
    
    
def softmax(logits):
    exp_logits = np.exp(logits)
    sum_logits = np.sum(exp_logits, axis=1, keepdims=True)
    return exp_logits / sum_logits


eta = 0.1
iteration = 5001
epsilon = 1e-7
theta = np.random.randn(X_train.shape[1], len(np.unique(y_raw)))

for i in range(iteration):
    logit = X_train.dot(theta)
    temp_predict = softmax(logit)
    loss = x_entropy(temp_predict, y_train_one_hot, epsilon)
    if (i % 500 == 0):
        print(i, loss)
    gradient = X_train.T.dot(temp_predict - y_train_one_hot) / len(X_train)
    theta = theta - eta * gradient

0 4.780427841483177
500 0.3669964205865573
1000 0.27158782196900244
1500 0.22385002416317776
2000 0.19344530661380266
2500 0.17189625674099207
3000 0.15564732311116947
3500 0.14287665308568848
4000 0.13253236325315337
4500 0.12395690182130371
5000 0.11671505661683196


In [5]:
theta

array([[ 8.62320594,  1.62993764, -8.91799214],
       [-1.51330669,  0.22919687,  0.22075157],
       [-4.8946616 , -1.50048214,  5.02367567]])

In [6]:
y_val_scores = softmax(X_val.dot(theta))
y_val_predict = np.argmax(y_val_scores, axis = 1)
accuracy_score = np.mean(y_val_predict == y_val)
print(accuracy_score)

0.9


#### Add Ridge Regularization and Early Stop

In [7]:
eta = 0.1
iteration = 35001
epsilon = 1e-7
theta = np.random.randn(X_train.shape[1], len(np.unique(y_raw)))
alpha = 0.05
last_loss = np.infty
early_stop_diff = 1e-8

for i in range(iteration):
    logit = X_train.dot(theta)
    temp_predict = softmax(logit)
    l2_loss = 0.5 * np.sum(np.square(theta[1:]))
    loss = -np.mean(np.sum(y_train_one_hot * np.log(temp_predict + epsilon), axis=1)) + alpha * l2_loss
    if (i % 500 == 0):
        print(i, loss)
    
    if last_loss - loss >= early_stop_diff:
        last_loss = loss
    else:
        print('Early stopping!')
        print('Last Run: ', i - 1, last_loss)
        print('Current Run: ', i, loss)
        break
    gradient = X_train.T.dot(temp_predict - y_train_one_hot) / len(X_train) + np.r_[np.zeros([1, len(np.unique(y_raw))]), alpha * theta[1:]]
    theta = theta - eta * gradient

0 1.8185811819995337
500 0.4720227487010162
1000 0.4201609814645636
1500 0.4000046231997925
2000 0.3900474991943235
2500 0.38467322237533774
3000 0.3816325829880317
3500 0.3798606486563636
4000 0.3788070488076408
4500 0.37817150295300495
5000 0.37778405943908144
5500 0.37754598498418634
6000 0.3773988093217512
6500 0.3773074039728006
7000 0.3772504314485231
7500 0.3772148213776322
8000 0.377192514979669
8500 0.377178518105095
9000 0.37716972344690225
Early stopping!
Last Run:  9349 0.37716560144117117
Current Run:  9350 0.3771655914413883


In [8]:
y_val_scores = softmax(X_val.dot(theta))
y_val_predict = np.argmax(y_val_scores, axis = 1)
accuracy_score = np.mean(y_val_predict == y_val)
print(accuracy_score)

0.9


#### Check Result on Test Set 

In [9]:
y_test_scores = softmax(X_test.dot(theta))
y_test_predict = np.argmax(y_test_scores, axis = 1)
accuracy_score = np.mean(y_test_predict == y_test)
print(accuracy_score)

1.0
